In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv(r"D:\DATA SCIENCE AND ML\Project\job_trend_predictor\data\processed\validated\jobs_validated_20260107_095333.csv")

In [3]:
df.columns

Index(['Title', 'Company', 'Experience', 'Salary', 'Location', 'Education',
       'Star_Skills', 'Normal_Skills', 'Posted_Date', 'Last_Apply_Date',
       'Role', 'Industry Type', 'Department', 'Employment Type',
       'Role Category', 'Description', 'Job_URL', 'Scraped_At', 'Job_Type'],
      dtype='object')

In [10]:
# updated code 
import re
from fuzzywuzzy import fuzz

unique_job_list = [
    "Data Scientist",
    "Machine Learning Engineer",
    "Data Analyst",
    "Business Analyst",
    "AI Engineer",
    "Research Associate",
    "Data Engineer",
    "Web Developer",
    "Software Engineer",
    "DevOps Engineer",
    "Product Manager",
    "Project Manager",
    "UX Designer",
    "Full Stack Developer",
    "Cloud Engineer",
    "DevOps Specialist",
    "Database Administrator",
    "Cybersecurity Analyst",
    "Network Engineer",
    "Systems Analyst",
    "IT Support Specialist",
    "Mobile App Developer",
    "Front End Developer",
    "Back End Developer",
    "QA Engineer",
    "Technical Writer",
    "Scrum Master",
    "Python Developer",
    "Java Developer",
    "Ruby on Rails Developer",
    "Big Data Engineer",
    "Data Architect",
    "AI Researcher",
    "Blockchain Developer",
    "Computer Vision Engineer",
    "NLP Engineer",
    "Robotics Engineer",
    "Game Developer",
    "Embedded Systems Engineer",
    "Site Reliability Engineer",
    "Solutions Architect",
    "IT Consultant",
    "Business Intelligence Analyst",
    "Generative AI Specialist",
    "Generative AI Engineer",
    "Technical Support Specialist",
    "Customer Support Specialist"
]

def fuzzy_matcher(job_list, x):
    """
    Match job title to predefined job list using fuzzy matching.
    
    Args:
        job_list: List of standardized job titles
        x: Raw job title string to match
        
    Returns:
        List containing the best matching job title(s) or ["Other"] if no match
    """
    if x is None:
        return ["Other"]
    
    if not job_list:
        raise ValueError("Please provide valid list of jobs")
    
    # Convert to lowercase and strip whitespace
    x = x.lower().strip()
    
    # Remove metadata keywords using word boundaries
    metadata_patterns = [
        r'\bhiring\b', r'\bimmediate\s+joiner?\b', r'\bopening\s+for\b',
        r'\burgent\b', r'\bimmediate\s+joining\b', r'\bwalk-?in\b',
        r'\bfreshers?\b', r'\bexperienced?\b'
    ]
    for pattern in metadata_patterns:
        x = re.sub(pattern, '', x, flags=re.IGNORECASE).strip()
    
    # Remove location markers (before first separator)
    x = re.split(r'\s*[-|,@]\s*', x)[0].strip()
    
    # Expand abbreviations with word boundaries for precision
    # Order matters: longer patterns first to avoid partial replacements
    x = re.sub(r'\bgen\s*ai\b', 'generative ai', x, flags=re.IGNORECASE)
    x = re.sub(r'\bsre\b', 'site reliability engineer', x, flags=re.IGNORECASE)
    x = re.sub(r'\bml\b', 'machine learning', x, flags=re.IGNORECASE)
    x = re.sub(r'\bai\b', 'artificial intelligence', x, flags=re.IGNORECASE)
    x = re.sub(r'\bfs\b', 'full stack', x, flags=re.IGNORECASE)
    x = re.sub(r'\bdev\b', 'developer', x, flags=re.IGNORECASE)
    x = re.sub(r'\bengg?\b', 'engineer', x, flags=re.IGNORECASE)
    x = re.sub(r'\bmgr\b', 'manager', x, flags=re.IGNORECASE)
    
    # Remove seniority markers with word boundaries
    seniority_patterns = [
        r'\bsenior\b', r'\bjr\.?\b', r'\bsr\.?\b', r'\blead\b',
        r'\bprincipal\b', r'\bassociate\b', r'\bintern\b', r'\btrainee\b',
        r'\bmid-?level\b', r'\bentry-?level\b', r'\bstaff\b'
    ]
    for pattern in seniority_patterns:
        x = re.sub(pattern, '', x, flags=re.IGNORECASE).strip()
    
    # Remove extra whitespace
    x = re.sub(r'\s+', ' ', x).strip()
    
    # If empty after cleaning, return Other
    if not x:
        return ["Other"]
    
    # Calculate fuzzy match scores
    matching_scores = {}
    for job in job_list:
        matching_scores[job] = fuzz.token_set_ratio(job.lower(), x)
    
    # Sort by score descending
    sorted_matches = sorted(matching_scores.items(), key=lambda item: item[1], reverse=True)
    
    # Get top score
    top_score = sorted_matches[0][1]
    
    # Return matches based on score thresholds
    if top_score >= 80:
        # High confidence: return top match only
        return [sorted_matches[0][0]]
    elif top_score >= 70:
        # Medium confidence: return top 2 if second score is within 5 points
        top_matches = [sorted_matches[0][0]]
        if len(sorted_matches) > 1 and sorted_matches[1][1] >= top_score - 5:
            top_matches.append(sorted_matches[1][0])
        return top_matches
    else:
        # Low confidence: return "Other"
        return ["Other"]


# Test cases
print("1.", fuzzy_matcher(unique_job_list, "Machine Learning Engineer and Data Scientist"))
print("2.", fuzzy_matcher(unique_job_list, "Frontend developer - SE - Noida"))
print("3.", fuzzy_matcher(unique_job_list, "Gen Ai Developer"))
print("4.", fuzzy_matcher(unique_job_list, "DevOps/SRE Lead (Specialist)-Pune"))
print("5.", fuzzy_matcher(unique_job_list, "Gen AI/Agentic AI Engineers"))
print("6.", fuzzy_matcher(unique_job_list, "Gen AI Engineer"))
print("7.", fuzzy_matcher(unique_job_list, "Hiring Machine Learning Engineerr"))
print("8.", fuzzy_matcher(unique_job_list, "Immediat Hiring for Machine Learning Engineerr"))
print("9.", fuzzy_matcher(unique_job_list, "  Full stack Developer (React + Springboot)"))
print("10.", fuzzy_matcher(unique_job_list, " Opening For Ruby on rails Developer"))
print("11.", fuzzy_matcher(unique_job_list, " Hiring | Customer Service / Tech Support | Jaipur | 3 To 4 LPA"))
print("12.", fuzzy_matcher(unique_job_list, " Hiring For International Voice Process - AR Callers"))
print("13.", fuzzy_matcher(unique_job_list, " Python Software Developer- Bangalore (Pan India Infosys)"))
print("14.", fuzzy_matcher(unique_job_list, "Senior Software Engineer"))
print("15.", fuzzy_matcher(unique_job_list, "ML Engineer"))
print("16.", fuzzy_matcher(unique_job_list, "DevOps/Cloud Engineer"))
print("17.", fuzzy_matcher(unique_job_list, ""))
print("18.", fuzzy_matcher(unique_job_list, None))

1. ['Data Scientist']
2. ['Web Developer']
3. ['Web Developer']
4. ['DevOps Engineer']
5. ['Other']
6. ['Generative AI Engineer']
7. ['Machine Learning Engineer']
8. ['Machine Learning Engineer']
9. ['Full Stack Developer']
10. ['Ruby on Rails Developer']
11. ['Other']
12. ['Other']
13. ['Python Developer']
14. ['Software Engineer']
15. ['Machine Learning Engineer']
16. ['DevOps Engineer']
17. ['Other']
18. ['Other']


In [11]:
df["TItle_Cleaned"] = df["Title"].apply(lambda x: fuzzy_matcher(unique_job_list, x))

In [12]:
# updated code 
import re
import pandas as pd

def clean_company_name(company: str) -> str:
    """
    Clean company name by removing hiring metadata, legal suffixes, and normalizing format.
    
    Args:
        company: Raw company name string
        
    Returns:
        Cleaned company name or "Unknown" if invalid/empty
    """
    # Handle None or non-string inputs
    if not isinstance(company, str) or company is None or pd.isna(company):
        return "Unknown"
    
    # Strip initial whitespace
    company = company.strip()
    
    if not company:
        return "Unknown"
    
    # Step 1: Remove hiring/job posting prefixes (case-insensitive)
    prefix_patterns = [
        r'\bHiring\s+for\b',
        r'\bImmediate\s+Hiring\b',
        r'\bPosted\s+by:?\b',
        r'\bUrgent\s+Hiring\b',
        r'\bWalk-?in\b',
        r'\bOpening\s+for\b',
        r'\bNow\s+Hiring\b',
        r'\bJobs?\s+at\b',
        r'\bCareers?\s+at\b',
        r'\bHiring\s*@\b',
        r'\bJoin\b',
        r'\bApply\s+to\b',
        r'\bWork\s+at\b'
    ]
    
    for pattern in prefix_patterns:
        company = re.sub(pattern, '', company, flags=re.IGNORECASE).strip()
    
    # Step 2: Split on common separators and take first part (main company name)
    # This handles cases like "Amazon | Bangalore" or "Google - Urgent"
    company = re.split(r'\s*[-|:@]\s*', company)[0].strip()
    
    # Step 3: Remove legal entity suffixes (case-insensitive)
    suffix_patterns = [
        r'\bPvt\.?\s*Ltd\.?\b',
        r'\bPrivate\s+Limited\b',
        r'\bLimited\b',
        r'\bLtd\.?\b',
        r'\bLLP\b',
        r'\bInc\.?\b',
        r'\bCorporation\b',
        r'\bCorp\.?\b',
        r'\bCompany\b',
        r'\bCo\.?\b(?!\w)',  # Only match "Co" if not part of another word
        r'\bPLC\b',
        r'\bLLC\b'
    ]
    
    for pattern in suffix_patterns:
        company = re.sub(pattern, '', company, flags=re.IGNORECASE).strip()
    
    # Step 4: Remove trailing urgency/metadata suffixes
    urgency_patterns = [
        r'\(?\s*Urgent\s*\)?',
        r'\(?\s*Immediate\s+Joiner\s*\)?',
        r'\(?\s*Work\s+From\s+Home\s*\)?',
        r'\(?\s*WFH\s*\)?',
        r'\(?\s*Remote\s*\)?',
        r'\(?\s*Hybrid\s*\)?'
    ]
    
    for pattern in urgency_patterns:
        company = re.sub(pattern, '', company, flags=re.IGNORECASE).strip()
    
    # Step 5: Normalize whitespace (multiple spaces → single space)
    company = re.sub(r'\s+', ' ', company).strip()
    
    # Step 6: Remove leading/trailing punctuation that might remain
    company = re.sub(r'^[,;:\-|@\s]+|[,;:\-|@\s]+$', '', company).strip()
    
    # Step 7: Handle empty results
    if not company or len(company) < 2:
        return "Unknown"
    
    # Step 8: Title case for consistency (optional - comment out if you want original case)
    # company = company.title()
    
    return company


# Test cases
test_cases = [
    "Hiring for Google Inc - Urgent",
    "Posted by: Amazon | Bangalore",
    "Accenture Pvt Ltd",
    "Microsoft Corporation",
    "Wipro",
    "TCS - Immediate Joiner",
    "Opening for Infosys Limited",
    "Now Hiring @ Adobe Inc.",
    "Flipkart - Work From Home",
    "Urgent Hiring - Cognizant",
    "Apple Inc - Remote",
    "  IBM Corp  ",
    "Oracle Corporation (Urgent)",
    "",
    None,
    "A",
    "Posted by Deloitte | Hyderabad | 5 LPA",
    "Cygnus Professionals",
    "Centre for Computational Technologies (CCTech)",
    "Mapmyindia Pvt. Ltd.",
    "Sirahu Technologies Inc"
]

print("Testing Company Name Cleaner:")
print("-" * 60)
for test in test_cases:
    result = clean_company_name(test)
    print(f"Input:  {repr(test):<45} → Output: {result}")

# Example usage with DataFrame
df["Company_Cleaned"] = df["Company"].apply(clean_company_name)

Testing Company Name Cleaner:
------------------------------------------------------------
Input:  'Hiring for Google Inc - Urgent'              → Output: Google
Input:  'Posted by: Amazon | Bangalore'               → Output: Unknown
Input:  'Accenture Pvt Ltd'                           → Output: Accenture
Input:  'Microsoft Corporation'                       → Output: Microsoft
Input:  'Wipro'                                       → Output: Wipro
Input:  'TCS - Immediate Joiner'                      → Output: TCS
Input:  'Opening for Infosys Limited'                 → Output: Infosys
Input:  'Now Hiring @ Adobe Inc.'                     → Output: Unknown
Input:  'Flipkart - Work From Home'                   → Output: Flipkart
Input:  'Urgent Hiring - Cognizant'                   → Output: Unknown
Input:  'Apple Inc - Remote'                          → Output: Apple
Input:  '  IBM Corp  '                                → Output: IBM
Input:  'Oracle Corporation (Urgent)'                

In [13]:
# updated code
import re
import pandas as pd
import numpy as np

# Define unexpected/invalid values at module level
INVALID_EXPERIENCE_VALUES = [
    "No fixed duration",
    "Not fixed",
    "Not Mentioned",
    "NA",
    "N/A",
    "TBD",
    "To be decided",
    "Negotiable"
]

def extract_experience_range(value: str) -> tuple:
    """
    Extract experience range from the given value.
    
    Args:
        value: The original experience value string
    
    Returns:
        Tuple of (min_exp, max_exp) as floats, or (None, None) if invalid
        
    Examples:
        "3-5 years" → (3.0, 5.0)
        "5 years" → (5.0, 5.0)
        "5+ years" → (5.0, None)
        "Fresher" → (0.0, 0.0)
        "No fixed duration" → (None, None)
    """
    # Handle non-string inputs
    if not isinstance(value, str) or pd.isna(value):
        return (None, None)
    
    # Clean and normalize input
    value = value.strip()
    
    if not value:
        return (None, None)
    
    # Check if value is in invalid list
    if value in INVALID_EXPERIENCE_VALUES:
        return (None, None)
    
    # Pattern 1: Fresher / Entry Level / No Experience
    fresher_pattern = r'\b(fresher|entry[\s-]?level|no\s+experience|0\s*years?)\b'
    if re.search(fresher_pattern, value, re.IGNORECASE):
        return (0.0, 0.0)
    
    # Pattern 2: Range with hyphen or "to" (e.g., "3-5 years", "3 to 5 yrs")
    range_pattern = r'(\d+\.?\d*)\s*(?:-|to)\s*(\d+\.?\d*)\s*(?:years?|yrs?)\b'
    match = re.search(range_pattern, value, re.IGNORECASE)
    if match:
        min_exp = float(match.group(1))
        max_exp = float(match.group(2))
        
        # Validate and swap if needed
        if min_exp > max_exp:
            min_exp, max_exp = max_exp, min_exp
        
        return (min_exp, max_exp)
    
    # Pattern 3: Plus sign (e.g., "5+ years", "3+ yrs")
    plus_pattern = r'(\d+\.?\d*)\s*\+\s*(?:years?|yrs?)\b'
    match = re.search(plus_pattern, value, re.IGNORECASE)
    if match:
        exp = float(match.group(1))
        return (exp, None)  # Use None for open-ended upper bound
    
    # Pattern 4: Single value (e.g., "5 years", "3 yrs")
    single_pattern = r'\b(\d+\.?\d*)\s*(?:years?|yrs?)\b'
    match = re.search(single_pattern, value, re.IGNORECASE)
    if match:
        exp = float(match.group(1))
        return (exp, exp)  # Single value means exact match
    
    # Pattern 5: Just a number (e.g., "5", "3.5")
    number_pattern = r'\b(\d+\.?\d*)\b'
    match = re.search(number_pattern, value, re.IGNORECASE)
    if match:
        exp = float(match.group(1))
        return (exp, exp)
    
    # No match found
    return (None, None)


def get_experience_min(experience_tuple: tuple) -> float:
    """Extract minimum experience from tuple."""
    if experience_tuple and experience_tuple[0] is not None:
        return experience_tuple[0]
    return np.nan


def get_experience_max(experience_tuple: tuple) -> float:
    """Extract maximum experience from tuple."""
    if experience_tuple and experience_tuple[1] is not None:
        return experience_tuple[1]
    return np.nan


# Test cases
test_cases = [
    "3-5 years",
    "5 years",
    "5+ years",
    "Fresher",
    "Entry Level",
    "0 years",
    "3 to 5 years",
    "2-3 yrs",
    "10 Years+",
    "No experience",
    "No fixed duration",
    "Not fixed",
    "NA",
    "5",
    "3.5 years",
    "1.5-2.5 years",
    "10-5 years",  # Reversed - should swap
    "  7 years  ",
    "",
    None,
    "Not Mentioned",
    "0-1 years",
    "15+ yrs"
]

print("Testing Experience Range Extraction:")
print("-" * 80)
for test in test_cases:
    result = extract_experience_range(test)
    min_exp = result[0] if result[0] is not None else "None"
    max_exp = result[1] if result[1] is not None else "None"
    print(f"Input: {repr(test):<25} → Output: ({min_exp}, {max_exp})")

print("\n" + "=" * 80)
print("Example DataFrame Usage:")
print("=" * 80)

# Example with DataFrame
sample_data = {
    'Experience': [
        "3-5 years",
        "5 years",
        "5+ years",
        "Fresher",
        "No fixed duration",
        "3 to 5 yrs",
        "0-1 years"
    ]
}

df_example = pd.DataFrame(sample_data)

# Apply the function
df_example['Experience_Range'] = df_example['Experience'].apply(extract_experience_range)
df_example['Min_Experience'] = df_example['Experience_Range'].apply(get_experience_min)
df_example['Max_Experience'] = df_example['Experience_Range'].apply(get_experience_max)

print(df_example)

print("\n" + "=" * 80)
print("Usage with your actual DataFrame:")
print("=" * 80)
print("""
# Apply to your DataFrame:
df['Experience_Range'] = df['Experience'].apply(extract_experience_range)
df['Min_Experience'] = df['Experience_Range'].apply(get_experience_min)
df['Max_Experience'] = df['Experience_Range'].apply(get_experience_max)

# Optional: Filter valid experiences
df_valid = df[df['Min_Experience'].notna()]

# Optional: Filter by experience range
df_mid_level = df[(df['Min_Experience'] >= 3) & (df['Max_Experience'] <= 7)]
""")

Testing Experience Range Extraction:
--------------------------------------------------------------------------------
Input: '3-5 years'               → Output: (3.0, 5.0)
Input: '5 years'                 → Output: (5.0, 5.0)
Input: '5+ years'                → Output: (5.0, None)
Input: 'Fresher'                 → Output: (0.0, 0.0)
Input: 'Entry Level'             → Output: (0.0, 0.0)
Input: '0 years'                 → Output: (0.0, 0.0)
Input: '3 to 5 years'            → Output: (3.0, 5.0)
Input: '2-3 yrs'                 → Output: (2.0, 3.0)
Input: '10 Years+'               → Output: (10.0, 10.0)
Input: 'No experience'           → Output: (0.0, 0.0)
Input: 'No fixed duration'       → Output: (None, None)
Input: 'Not fixed'               → Output: (None, None)
Input: 'NA'                      → Output: (None, None)
Input: '5'                       → Output: (5.0, 5.0)
Input: '3.5 years'               → Output: (3.5, 3.5)
Input: '1.5-2.5 years'           → Output: (1.5, 2.5)
Input: '1

In [14]:
df['Experience_Range'] = df['Experience'].apply(extract_experience_range)
df['Min_Experience'] = df['Experience_Range'].apply(get_experience_min)
df['Max_Experience'] = df['Experience_Range'].apply(get_experience_max)

In [15]:
# updated Code
import re
import pandas as pd
import numpy as np

# Define invalid/undisclosed salary values
INVALID_SALARY_VALUES = [
    "Not Disclosed",
    "Salary Not Disclosed",
    "NA",
    "N/A",
    "Not Mentioned",
    "Negotiable",
    "Competitive",
    "As per industry standards",
    "TBD",
    "To be decided"
]

def extract_salary_range(value: str) -> tuple:
    """
    Extract salary range from the given value and normalize to Lakhs Per Annum (LPA).
    
    Args:
        value: The original salary value string
    
    Returns:
        Tuple of (min_salary, max_salary) as floats in LPA, or (None, None) if invalid
        
    Examples:
        "3-5 LPA" → (3.0, 5.0)
        "5 Lakhs" → (5.0, 5.0)
        "Upto 7 LPA" → (0.0, 7.0)
        "5+ LPA" → (5.0, None)
        "500000-700000" → (5.0, 7.0)  # Converted to Lakhs
        "Not Disclosed" → (None, None)
    """
    # Handle non-string inputs
    if not isinstance(value, str) or pd.isna(value):
        return (None, None)
    
    # Clean and normalize input
    value = value.strip()
    
    if not value:
        return (None, None)
    
    # Check if value is in invalid/undisclosed list (case-insensitive)
    if any(value.lower() == invalid.lower() for invalid in INVALID_SALARY_VALUES):
        return (None, None)
    
    # Pattern 1: "Upto X" or "Up to X" (e.g., "Upto 5 LPA", "Up to 7 Lakhs")
    upto_pattern = r'(?:upto|up\s*to)\s*(\d+\.?\d*)\s*(?:lpa|l\.?p\.?a\.?|lakhs?|lacs?|l)\b'
    match = re.search(upto_pattern, value, re.IGNORECASE)
    if match:
        max_salary = float(match.group(1))
        return (0.0, max_salary)
    
    # Pattern 2: "X+" (e.g., "5+ LPA", "3+ Lakhs")
    plus_pattern = r'(\d+\.?\d*)\s*\+\s*(?:lpa|l\.?p\.?a\.?|lakhs?|lacs?|l)\b'
    match = re.search(plus_pattern, value, re.IGNORECASE)
    if match:
        min_salary = float(match.group(1))
        return (min_salary, None)
    
    # Pattern 3: Range with LPA/Lakhs/Lacs units (e.g., "3-5 LPA", "3 to 5 Lakhs")
    range_with_unit_pattern = r'(\d+\.?\d*)\s*(?:-|to)\s*(\d+\.?\d*)\s*(?:lpa|l\.?p\.?a\.?|lakhs?|lacs?|l)(?:\s*p\.?a\.?|\s*per\s*annum)?\b'
    match = re.search(range_with_unit_pattern, value, re.IGNORECASE)
    if match:
        min_salary = float(match.group(1))
        max_salary = float(match.group(2))
        
        # Validate and swap if needed
        if min_salary > max_salary:
            min_salary, max_salary = max_salary, min_salary
        
        return (min_salary, max_salary)
    
    # Pattern 4: Single value with LPA/Lakhs/Lacs (e.g., "5 LPA", "3.5 Lakhs")
    single_with_unit_pattern = r'(\d+\.?\d*)\s*(?:lpa|l\.?p\.?a\.?|lakhs?|lacs?|l)(?:\s*p\.?a\.?|\s*per\s*annum)?\b'
    match = re.search(single_with_unit_pattern, value, re.IGNORECASE)
    if match:
        salary = float(match.group(1))
        return (salary, salary)
    
    # Pattern 5: Absolute numbers range (e.g., "500000-700000", "50000 to 70000")
    # Convert to Lakhs (divide by 100,000)
    absolute_range_pattern = r'(\d{5,})\s*(?:-|to)\s*(\d{5,})'
    match = re.search(absolute_range_pattern, value, re.IGNORECASE)
    if match:
        min_salary = float(match.group(1)) / 100000  # Convert to Lakhs
        max_salary = float(match.group(2)) / 100000
        
        # Validate and swap if needed
        if min_salary > max_salary:
            min_salary, max_salary = max_salary, min_salary
        
        # Round to 2 decimal places
        return (round(min_salary, 2), round(max_salary, 2))
    
    # Pattern 6: Single absolute number (e.g., "600000", "500000")
    # Convert to Lakhs if >= 10000 (assume it's in rupees, not lakhs)
    absolute_single_pattern = r'(\d{5,})'
    match = re.search(absolute_single_pattern, value, re.IGNORECASE)
    if match:
        salary = float(match.group(1)) / 100000  # Convert to Lakhs
        salary = round(salary, 2)
        return (salary, salary)
    
    # Pattern 7: Just numbers with range (fallback, e.g., "3-5", "5 to 7")
    # Assume already in Lakhs if no unit specified and numbers are small
    simple_range_pattern = r'(\d+\.?\d*)\s*(?:-|to)\s*(\d+\.?\d*)'
    match = re.search(simple_range_pattern, value, re.IGNORECASE)
    if match:
        min_salary = float(match.group(1))
        max_salary = float(match.group(2))
        
        # Only accept if numbers are reasonable for Lakhs (< 100)
        if min_salary < 100 and max_salary < 100:
            if min_salary > max_salary:
                min_salary, max_salary = max_salary, min_salary
            return (min_salary, max_salary)
    
    # No match found
    return (None, None)


def get_salary_min(salary_tuple: tuple) -> float:
    """Extract minimum salary from tuple."""
    if salary_tuple and salary_tuple[0] is not None:
        return salary_tuple[0]
    return np.nan


def get_salary_max(salary_tuple: tuple) -> float:
    """Extract maximum salary from tuple."""
    if salary_tuple and salary_tuple[1] is not None:
        return salary_tuple[1]
    return np.nan


def get_salary_avg(salary_tuple: tuple) -> float:
    """Calculate average salary from tuple."""
    if salary_tuple and salary_tuple[0] is not None and salary_tuple[1] is not None:
        return round((salary_tuple[0] + salary_tuple[1]) / 2, 2)
    elif salary_tuple and salary_tuple[0] is not None:
        return salary_tuple[0]
    return np.nan


# Test cases
test_cases = [
    "3-5 LPA",
    "5-7 Lakhs PA",
    "3.5 Lacs",
    "Upto 5 LPA",
    "5+ LPA",
    "Not Disclosed",
    "500000-700000",
    "600000",
    "3 to 5 LPA",
    "5 L",
    "3-5 L.P.A",
    "7 Lakhs Per Annum",
    "Negotiable",
    "NA",
    "3.5-5.5 Lakhs",
    "Up to 10 LPA",
    "8+ Lakhs",
    "300000",
    "10-5 LPA",  # Reversed - should swap
    "  5 LPA  ",
    "",
    None,
    "As per industry standards",
    "1000000-1500000",  # 10-15 Lakhs
    "2.5 to 4 L"
]

print("Testing Salary Range Extraction:")
print("-" * 80)
for test in test_cases:
    result = extract_salary_range(test)
    min_sal = result[0] if result[0] is not None else "None"
    max_sal = result[1] if result[1] is not None else "None"
    print(f"Input: {repr(test):<35} → Output: ({min_sal}, {max_sal})")

print("\n" + "=" * 80)
print("Example DataFrame Usage:")
print("=" * 80)

# Example with DataFrame
sample_data = {
    'Salary': [
        "3-5 LPA",
        "5 Lakhs",
        "Upto 7 LPA",
        "5+ LPA",
        "Not Disclosed",
        "500000-700000",
        "3 to 5 Lakhs"
    ]
}

df_example = pd.DataFrame(sample_data)

# Apply the function
df_example['Salary_Range'] = df_example['Salary'].apply(extract_salary_range)
df_example['Min_Salary'] = df_example['Salary_Range'].apply(get_salary_min)
df_example['Max_Salary'] = df_example['Salary_Range'].apply(get_salary_max)
df_example['Avg_Salary'] = df_example['Salary_Range'].apply(get_salary_avg)

print(df_example)

print("\n" + "=" * 80)
print("Usage with your actual DataFrame:")
print("=" * 80)
print("""
# Apply to your DataFrame:
df['Salary_Range'] = df['Salary'].apply(extract_salary_range)
df['Min_Salary'] = df['Salary_Range'].apply(get_salary_min)
df['Max_Salary'] = df['Salary_Range'].apply(get_salary_max)
df['Avg_Salary'] = df['Salary_Range'].apply(get_salary_avg)

# Optional: Filter valid salaries
df_valid = df[df['Min_Salary'].notna()]

# Optional: Filter by salary range
df_mid_range = df[(df['Min_Salary'] >= 3) & (df['Max_Salary'] <= 10)]

# Optional: Find jobs with competitive pay (> 5 LPA average)
df_high_pay = df[df['Avg_Salary'] > 5]
""")

Testing Salary Range Extraction:
--------------------------------------------------------------------------------
Input: '3-5 LPA'                           → Output: (3.0, 5.0)
Input: '5-7 Lakhs PA'                      → Output: (5.0, 7.0)
Input: '3.5 Lacs'                          → Output: (3.5, 3.5)
Input: 'Upto 5 LPA'                        → Output: (0.0, 5.0)
Input: '5+ LPA'                            → Output: (5.0, None)
Input: 'Not Disclosed'                     → Output: (None, None)
Input: '500000-700000'                     → Output: (5.0, 7.0)
Input: '600000'                            → Output: (6.0, 6.0)
Input: '3 to 5 LPA'                        → Output: (3.0, 5.0)
Input: '5 L'                               → Output: (5.0, 5.0)
Input: '3-5 L.P.A'                         → Output: (3.0, 5.0)
Input: '7 Lakhs Per Annum'                 → Output: (7.0, 7.0)
Input: 'Negotiable'                        → Output: (None, None)
Input: 'NA'                                → Outp

In [16]:
df['Salary_Range'] = df['Salary'].apply(extract_salary_range)
df['Min_Salary'] = df['Salary_Range'].apply(get_salary_min)
df['Max_Salary'] = df['Salary_Range'].apply(get_salary_max)
df['Avg_Salary'] = df['Salary_Range'].apply(get_salary_avg)

In [17]:
import re
import pandas as pd
import numpy as np

def clean_education(value: str) -> dict:
    """
    Clean education value by extracting and normalizing UG and PG details.
    
    Args:
        value: Original education string (e.g., "UG: B.Tech/B.E. in Computer Science, PG: M.Tech in AI")
    
    Returns:
        Dictionary with keys: UG_Degree, UG_Specialization, PG_Degree, PG_Specialization
        
    Examples:
        "UG: B.Tech/B.E. in Computer Science" 
        → {"UG_Degree": "B.Tech", "UG_Specialization": "Computer Science", 
           "PG_Degree": None, "PG_Specialization": None}
    """
    # Default structure
    result = {
        "UG_Degree": None,
        "UG_Specialization": None,
        "PG_Degree": None,
        "PG_Specialization": None
    }
    
    # Handle invalid inputs
    if not isinstance(value, str) or pd.isna(value) or not value.strip():
        return result
    
    value = value.strip()
    
    # Check for "Not Required" patterns
    if re.search(r'\b(not required|not specified|any|na|n/a)\b', value, re.IGNORECASE):
        if "UG" in value:
            result["UG_Degree"] = "Not Required"
        if "PG" in value:
            result["PG_Degree"] = "Not Required"
        return result
    
    # Extract UG and PG sections
    pattern = r'(UG|PG):\s*(.*?)(?=\s*(?:UG|PG):|$)'
    matches = re.findall(pattern, value, re.IGNORECASE)
    
    for level, content in matches:
        level = level.upper()
        content = content.strip().rstrip(',').strip()
        
        if not content:
            continue
        
        # Check for "Any Graduate" pattern
        if re.search(r'\bany\s+(graduate|postgraduate)\b', content, re.IGNORECASE):
            if level == "UG":
                result["UG_Degree"] = "Any Graduate"
            else:
                result["PG_Degree"] = "Any Postgraduate"
            continue
        
        # Extract degree and specialization
        # Pattern: "B.Tech/B.E. in Computer Science" or "M.Tech in AI"
        degree_spec_pattern = r'([A-Za-z./]+(?:\s*\/\s*[A-Za-z./]+)*)\s+in\s+(.+)'
        match = re.search(degree_spec_pattern, content, re.IGNORECASE)
        
        if match:
            degree = match.group(1).strip()
            specialization = match.group(2).strip()
            
            # Normalize degree
            degree = normalize_degree(degree)
            
            if level == "UG":
                result["UG_Degree"] = degree
                result["UG_Specialization"] = specialization
            else:
                result["PG_Degree"] = degree
                result["PG_Specialization"] = specialization
        else:
            # No "in" keyword - treat entire content as degree
            degree = normalize_degree(content)
            
            if level == "UG":
                result["UG_Degree"] = degree
                result["UG_Specialization"] = "Any Specialization"
            else:
                result["PG_Degree"] = degree
                result["PG_Specialization"] = "Any Specialization"
    
    return result


def normalize_degree(degree: str) -> str:
    """
    Normalize degree names to standard format.
    
    Examples:
        "B.Tech/B.E." → "B.Tech"
        "Bachelor of Technology" → "B.Tech"
        "M.Tech" → "M.Tech"
    """
    degree = degree.strip()
    
    # Common degree mappings
    degree_map = {
        # Undergraduate
        r'\b(b\.?tech|b\.?e\.?|bachelor\s+of\s+technology|bachelor\s+of\s+engineering)\b': 'B.Tech',
        r'\b(bca|bachelor\s+of\s+computer\s+applications?)\b': 'BCA',
        r'\b(bsc|b\.?sc\.?|bachelor\s+of\s+science)\b': 'B.Sc',
        r'\b(bcom|b\.?com\.?|bachelor\s+of\s+commerce)\b': 'B.Com',
        r'\b(ba|b\.?a\.?|bachelor\s+of\s+arts)\b': 'B.A',
        r'\b(bba|bachelor\s+of\s+business\s+administration)\b': 'BBA',
        r'\b(diploma)\b': 'Diploma',
        
        # Postgraduate
        r'\b(m\.?tech|m\.?e\.?|master\s+of\s+technology|master\s+of\s+engineering)\b': 'M.Tech',
        r'\b(mca|master\s+of\s+computer\s+applications?)\b': 'MCA',
        r'\b(msc|m\.?sc\.?|master\s+of\s+science)\b': 'M.Sc',
        r'\b(mba|master\s+of\s+business\s+administration)\b': 'MBA',
        r'\b(ma|m\.?a\.?|master\s+of\s+arts)\b': 'M.A',
        r'\b(phd|ph\.?d\.?|doctorate)\b': 'Ph.D',
        r'\b(mcom|m\.?com\.?|master\s+of\s+commerce)\b': 'M.Com',
    }
    
    for pattern, normalized in degree_map.items():
        if re.search(pattern, degree, re.IGNORECASE):
            return normalized
    
    # If no match, return cleaned version
    return degree.replace('/', ' ').strip()


# Test cases
test_cases = [
    "UG: B.Tech/B.E. in Computer Science, PG: M.Tech in Artificial Intelligence",
    "UG: Any Graduate, PG: Any Postgraduate",
    "UG: B.Tech/B.E. in Mechanical",
    "UG: BCA in Computer Applications",
    "UG: B.Sc in Physics, PG: M.Sc in Data Science",
    "UG: Bachelor of Technology in ECE",
    "UG: Not Required, PG: MBA",
    "UG: Diploma in Engineering",
    "PG: Ph.D in Machine Learning",
    "",
    None,
    "UG: B.Tech/B.E. in Any Specialization, PG: M.Tech in VLSI",
    "UG: Any Graduate",
    "UG: BBA, PG: MBA in Finance"
]

print("Testing Education Cleaning:")
print("-" * 100)
for test in test_cases:
    result = clean_education(test)
    print(f"Input: {repr(test):<70}")
    print(f"  UG: {result['UG_Degree']} | {result['UG_Specialization']}")
    print(f"  PG: {result['PG_Degree']} | {result['PG_Specialization']}")
    print()

print("=" * 100)
print("Example DataFrame Usage:")
print("=" * 100)

# Example with DataFrame
sample_data = {
    'Education': [
        "UG: B.Tech/B.E. in Computer Science, PG: M.Tech in AI",
        "UG: Any Graduate, PG: Any Postgraduate",
        "UG: BCA in Computer Applications",
        "UG: Not Required, PG: MBA",
        None
    ]
}

df_example = pd.DataFrame(sample_data)

# Apply the function
df_example['Education_Cleaned'] = df_example['Education'].apply(clean_education)

# Extract into separate columns
df_example['UG_Degree'] = df_example['Education_Cleaned'].apply(lambda x: x['UG_Degree'])
df_example['UG_Specialization'] = df_example['Education_Cleaned'].apply(lambda x: x['UG_Specialization'])
df_example['PG_Degree'] = df_example['Education_Cleaned'].apply(lambda x: x['PG_Degree'])
df_example['PG_Specialization'] = df_example['Education_Cleaned'].apply(lambda x: x['PG_Specialization'])

print(df_example[['Education', 'UG_Degree', 'UG_Specialization', 'PG_Degree', 'PG_Specialization']])

print("\n" + "=" * 100)
print("Recommended Usage:")
print("=" * 100)
print("""
# Apply cleaning
df['Education_Cleaned'] = df['Education'].apply(clean_education)

# Extract to separate columns for easy filtering
df['UG_Degree'] = df['Education_Cleaned'].apply(lambda x: x['UG_Degree'])
df['UG_Specialization'] = df['Education_Cleaned'].apply(lambda x: x['UG_Specialization'])
df['PG_Degree'] = df['Education_Cleaned'].apply(lambda x: x['PG_Degree'])
df['PG_Specialization'] = df['Education_Cleaned'].apply(lambda x: x['PG_Specialization'])

# Example queries:
# Find all B.Tech Computer Science jobs
df_cs_jobs = df[df['UG_Specialization'].str.contains('Computer', na=False)]

# Find jobs requiring PG degree
df_pg_required = df[df['PG_Degree'].notna() & (df['PG_Degree'] != 'Not Required')]

# Count by degree type
df['UG_Degree'].value_counts()
""")

Testing Education Cleaning:
----------------------------------------------------------------------------------------------------
Input: 'UG: B.Tech/B.E. in Computer Science, PG: M.Tech in Artificial Intelligence'
  UG: B.Tech | Computer Science
  PG: M.Tech | Artificial Intelligence

Input: 'UG: Any Graduate, PG: Any Postgraduate'                              
  UG: Not Required | None
  PG: Not Required | None

Input: 'UG: B.Tech/B.E. in Mechanical'                                       
  UG: B.Tech | Mechanical
  PG: None | None

Input: 'UG: BCA in Computer Applications'                                    
  UG: BCA | Computer Applications
  PG: None | None

Input: 'UG: B.Sc in Physics, PG: M.Sc in Data Science'                       
  UG: B.Sc | Physics
  PG: M.Sc | Data Science

Input: 'UG: Bachelor of Technology in ECE'                                   
  UG: Technology | ECE
  PG: None | None

Input: 'UG: Not Required, PG: MBA'                                           
  UG: 

In [19]:

df['Education_Cleaned'] = df['Education'].apply(clean_education)

df['UG_Degree'] = df['Education_Cleaned'].apply(lambda x: x['UG_Degree'])
df['UG_Specialization'] = df['Education_Cleaned'].apply(lambda x: x['UG_Specialization'])
df['PG_Degree'] = df['Education_Cleaned'].apply(lambda x: x['PG_Degree'])
df['PG_Specialization'] = df['Education_Cleaned'].apply(lambda x: x['PG_Specialization'])

In [20]:
# updated code:
import re
import pandas as pd
from typing import Union, List

# Skill normalization mapping (abbreviations and synonyms)
SKILL_NORMALIZATION = {
    # Programming Languages
    'js': 'JavaScript',
    'javascript': 'JavaScript',
    'reactjs': 'React',
    'react.js': 'React',
    'nodejs': 'Node.js',
    'node.js': 'Node.js',
    'node': 'Node.js',
    'ts': 'TypeScript',
    'typescript': 'TypeScript',
    'py': 'Python',
    'python': 'Python',
    'c++': 'C++',
    'cpp': 'C++',
    'c#': 'C#',
    'csharp': 'C#',
    'golang': 'Go',
    
    # Frameworks & Libraries
    'angular.js': 'Angular',
    'angularjs': 'Angular',
    'vue.js': 'Vue',
    'vuejs': 'Vue',
    'nextjs': 'Next.js',
    'next.js': 'Next.js',
    'expressjs': 'Express.js',
    'express.js': 'Express.js',
    'django': 'Django',
    'flask': 'Flask',
    'springboot': 'Spring Boot',
    'spring boot': 'Spring Boot',
    
    # Data Science & ML
    'ml': 'Machine Learning',
    'machine learning': 'Machine Learning',
    'ai': 'Artificial Intelligence',
    'artificial intelligence': 'Artificial Intelligence',
    'dl': 'Deep Learning',
    'deep learning': 'Deep Learning',
    'nlp': 'Natural Language Processing',
    'cv': 'Computer Vision',
    'computer vision': 'Computer Vision',
    'tensorflow': 'TensorFlow',
    'pytorch': 'PyTorch',
    'scikit-learn': 'Scikit-learn',
    'sklearn': 'Scikit-learn',
    'pandas': 'Pandas',
    'numpy': 'NumPy',
    
    # Databases
    'sql': 'SQL',
    'mysql': 'MySQL',
    'postgresql': 'PostgreSQL',
    'postgres': 'PostgreSQL',
    'mongodb': 'MongoDB',
    'mongo': 'MongoDB',
    'nosql': 'NoSQL',
    'redis': 'Redis',
    
    # Cloud & DevOps
    'aws': 'AWS',
    'amazon web services': 'AWS',
    'azure': 'Azure',
    'gcp': 'Google Cloud Platform',
    'google cloud': 'Google Cloud Platform',
    'docker': 'Docker',
    'kubernetes': 'Kubernetes',
    'k8s': 'Kubernetes',
    'ci/cd': 'CI/CD',
    'jenkins': 'Jenkins',
    'git': 'Git',
    'github': 'GitHub',
    'gitlab': 'GitLab',
    
    # Other
    'api': 'API',
    'rest api': 'REST API',
    'restful': 'REST API',
    'graphql': 'GraphQL',
    'html': 'HTML',
    'css': 'CSS',
    'sass': 'SASS',
    'scss': 'SCSS',
}

# Skills to exclude (noise/invalid)
INVALID_SKILLS = {
    'etc', 'and', 'or', 'with', 'using', 'knowledge', 'experience',
    'skills', 'good', 'strong', 'excellent', 'basic', 'advanced',
    'familiarity', 'understanding', 'ability', 'na', 'n/a', 'none',
    'required', 'preferred', 'must', 'have', 'plus', 'bonus'
}


def normalize_skill(skill: str) -> Union[str, None]:
    """
    Normalize a single skill name.
    
    Args:
        skill: Raw skill string
        
    Returns:
        Normalized skill name or None if invalid
    """
    if not skill or not isinstance(skill, str):
        return None
    
    # Clean whitespace and convert to lowercase for matching
    skill = skill.strip().lower()
    
    # Remove common punctuation at edges
    skill = re.sub(r'^[.,;:\-/|()]+|[.,;:\-/|()]+$', '', skill).strip()
    
    # Skip if empty after cleaning
    if not skill or len(skill) < 2:
        return None
    
    # Skip if in invalid list
    if skill in INVALID_SKILLS:
        return None
    
    # Skip single letters (unless known abbreviations like 'R', 'C')
    if len(skill) == 1 and skill not in ['r', 'c']:
        return None
    
    # Normalize using mapping
    if skill in SKILL_NORMALIZATION:
        return SKILL_NORMALIZATION[skill]
    
    # Return title-cased version if not in mapping
    return skill.title()


def parse_skills_string(skills_str: Union[str, List]) -> List[str]:
    """
    Parse skills from string or list format.
    
    Args:
        skills_str: Skills as string (comma/pipe/semicolon separated) or list
        
    Returns:
        List of individual skill strings
    """
    # Handle None or empty
    if pd.isna(skills_str) or not skills_str:
        return []
    
    # If already a list
    if isinstance(skills_str, list):
        return [str(s).strip() for s in skills_str if s]
    
    # Convert to string if not already
    skills_str = str(skills_str).strip()
    
    # Remove surrounding brackets if present (e.g., "[Python, SQL]")
    skills_str = re.sub(r'^\[|\]$', '', skills_str)
    
    # Remove quotes that might wrap skills
    skills_str = re.sub(r'["\']', '', skills_str)
    
    # Split on multiple possible separators: comma, pipe, semicolon, forward slash
    skills_list = re.split(r'[,|;/]', skills_str)
    
    # Clean each skill
    skills_list = [s.strip() for s in skills_list if s and s.strip()]
    
    return skills_list


def get_combined_skills(star_skills: Union[str, List], 
                        normal_skills: Union[str, List],
                        preserve_order: bool = True) -> List[str]:
    """
    Combine and normalize star skills and normal skills.
    
    Args:
        star_skills: Primary/featured skills (string or list)
        normal_skills: Additional skills (string or list)
        preserve_order: If True, star skills appear first (default: True)
        
    Returns:
        List of unique, normalized skills
        
    Examples:
        star_skills = "Python, ML, JS"
        normal_skills = "SQL, python, machine learning"
        → ["Python", "Machine Learning", "JavaScript", "SQL"]
    """
    # Parse skills from both sources
    star_list = parse_skills_string(star_skills)
    normal_list = parse_skills_string(normal_skills)
    
    # Normalize all skills
    normalized_star = [normalize_skill(s) for s in star_list]
    normalized_normal = [normalize_skill(s) for s in normal_list]
    
    # Remove None values
    normalized_star = [s for s in normalized_star if s is not None]
    normalized_normal = [s for s in normalized_normal if s is not None]
    
    if preserve_order:
        # Keep star skills first, then add normal skills not already present
        # Use dict to maintain order while deduplicating (Python 3.7+)
        seen = {}
        for skill in normalized_star:
            seen[skill] = None
        for skill in normalized_normal:
            if skill not in seen:
                seen[skill] = None
        return list(seen.keys())
    else:
        # Just deduplicate without preserving order
        return list(set(normalized_star + normalized_normal))


def get_skill_count(skills_list: List[str]) -> int:
    """Get count of skills in list."""
    return len(skills_list) if isinstance(skills_list, list) else 0


def has_skill(skills_list: List[str], target_skill: str) -> bool:
    """Check if target skill is in the skills list (case-insensitive)."""
    if not isinstance(skills_list, list):
        return False
    target_normalized = normalize_skill(target_skill)
    return target_normalized in skills_list if target_normalized else False


# Test cases
test_cases = [
    # (star_skills, normal_skills)
    ("Python, ML, JS", "SQL, python, machine learning"),
    ("[React, Node.js]", "JavaScript, reactjs, nodejs"),
    ("AWS, Docker, K8s", "Kubernetes, aws, docker, CI/CD"),
    ("", "Python, Java, C++"),
    (None, "Machine Learning, Deep Learning"),
    ("Python", None),
    ("Python, etc, and, SQL", "good knowledge of ML"),
    ("TensorFlow, PyTorch", "tensorflow, pytorch, pandas"),
    ("HTML/CSS/JavaScript", "React|Angular|Vue"),
    ("'Python', 'SQL'", "['AWS', 'Docker']"),
]

print("Testing Skills Cleaning:")
print("-" * 100)
for star, normal in test_cases:
    result = get_combined_skills(star, normal)
    print(f"Star:   {repr(star):<40}")
    print(f"Normal: {repr(normal):<40}")
    print(f"Result: {result}")
    print(f"Count:  {len(result)} skills")
    print()

print("=" * 100)
print("Example DataFrame Usage:")
print("=" * 100)

# Example with DataFrame
sample_data = {
    'Star_Skills': [
        "Python, ML, JS",
        "[React, Node.js]",
        "AWS, Docker, K8s",
        None,
        "TensorFlow, PyTorch"
    ],
    'Normal_Skills': [
        "SQL, python, machine learning",
        "JavaScript, reactjs, nodejs",
        "Kubernetes, aws, docker, CI/CD",
        "Python, Java, C++",
        "tensorflow, pytorch, pandas"
    ]
}

df_example = pd.DataFrame(sample_data)

# Apply cleaning
df_example['Combined_Skills'] = df_example.apply(
    lambda row: get_combined_skills(row['Star_Skills'], row['Normal_Skills']), 
    axis=1
)

df_example['Skill_Count'] = df_example['Combined_Skills'].apply(get_skill_count)

# Check for specific skills
df_example['Has_Python'] = df_example['Combined_Skills'].apply(lambda x: has_skill(x, 'Python'))
df_example['Has_ML'] = df_example['Combined_Skills'].apply(lambda x: has_skill(x, 'Machine Learning'))

print(df_example[['Star_Skills', 'Normal_Skills', 'Combined_Skills', 'Skill_Count']])

print("\n" + "=" * 100)
print("Recommended Usage:")
print("=" * 100)
print("""
# Apply skills cleaning
df['Combined_Skills'] = df.apply(
    lambda row: get_combined_skills(row['Star_Skills'], row['Normal_Skills']),
    axis=1
)

df['Skill_Count'] = df['Combined_Skills'].apply(get_skill_count)

# Find jobs requiring specific skills
df['Requires_Python'] = df['Combined_Skills'].apply(lambda x: has_skill(x, 'Python'))
df['Requires_ML'] = df['Combined_Skills'].apply(lambda x: 'Machine Learning' in x if isinstance(x, list) else False)

# Filter jobs by skills
python_jobs = df[df['Requires_Python']]
ml_jobs = df[df['Requires_ML']]

# Most common skills (flatten all skill lists)
from collections import Counter
all_skills = [skill for skills in df['Combined_Skills'] for skill in skills]
skill_frequency = Counter(all_skills)
print(skill_frequency.most_common(20))

# Skills co-occurrence analysis
# Find what skills appear together with Python
python_jobs_skills = df[df['Requires_Python']]['Combined_Skills']
python_cooccurrence = Counter([s for skills in python_jobs_skills for s in skills if s != 'Python'])
""")

Testing Skills Cleaning:
----------------------------------------------------------------------------------------------------
Star:   'Python, ML, JS'                        
Normal: 'SQL, python, machine learning'         
Result: ['Python', 'Machine Learning', 'JavaScript', 'SQL']
Count:  4 skills

Star:   '[React, Node.js]'                      
Normal: 'JavaScript, reactjs, nodejs'           
Result: ['React', 'Node.js', 'JavaScript']
Count:  3 skills

Star:   'AWS, Docker, K8s'                      
Normal: 'Kubernetes, aws, docker, CI/CD'        
Result: ['AWS', 'Docker', 'Kubernetes', 'Ci', 'Cd']
Count:  5 skills

Star:   ''                                      
Normal: 'Python, Java, C++'                     
Result: ['Python', 'Java', 'C++']
Count:  3 skills

Star:   None                                    
Normal: 'Machine Learning, Deep Learning'       
Result: ['Machine Learning', 'Deep Learning']
Count:  2 skills

Star:   'Python'                                
Normal: No

In [21]:
df['Combined_Skills'] = df.apply(
    lambda row: get_combined_skills(row['Star_Skills'], row['Normal_Skills']),
    axis=1
)

df['Skill_Count'] = df['Combined_Skills'].apply(get_skill_count)

In [22]:
# updated code:
import re
import pandas as pd
from typing import List, Union

# Department normalization mapping
DEPARTMENT_NORMALIZATION = {
    # IT & Technology
    'it': 'Information Technology',
    'information technology': 'Information Technology',
    'tech': 'Technology',
    'technology': 'Technology',
    'software': 'Software Development',
    'software development': 'Software Development',
    'software engineering': 'Software Engineering',
    'engineering': 'Engineering',
    'data': 'Data Science',
    'data science': 'Data Science',
    'analytics': 'Analytics',
    'ai': 'Artificial Intelligence',
    'ml': 'Machine Learning',
    
    # Business Functions
    'hr': 'Human Resources',
    'human resources': 'Human Resources',
    'human resource': 'Human Resources',
    'finance': 'Finance',
    'accounts': 'Finance',
    'accounting': 'Finance',
    'sales': 'Sales',
    'marketing': 'Marketing',
    'sales & marketing': 'Sales & Marketing',
    'sales and marketing': 'Sales & Marketing',
    'operations': 'Operations',
    'ops': 'Operations',
    'admin': 'Administration',
    'administration': 'Administration',
    'legal': 'Legal',
    
    # R&D and Product
    'r&d': 'Research & Development',
    'research': 'Research & Development',
    'research and development': 'Research & Development',
    'product': 'Product Management',
    'product management': 'Product Management',
    'project management': 'Project Management',
    'pmo': 'Project Management Office',
    
    # Operations & Support
    'customer service': 'Customer Service',
    'customer support': 'Customer Support',
    'support': 'Customer Support',
    'technical support': 'Technical Support',
    'quality': 'Quality Assurance',
    'qa': 'Quality Assurance',
    'quality assurance': 'Quality Assurance',
    'testing': 'Quality Assurance',
    
    # Design & Creative
    'design': 'Design',
    'ui/ux': 'UI/UX Design',
    'ux': 'UX Design',
    'ui': 'UI Design',
    'creative': 'Creative',
    'graphics': 'Graphics Design',
    
    # Infrastructure
    'devops': 'DevOps',
    'infrastructure': 'Infrastructure',
    'network': 'Network',
    'security': 'Security',
    'cybersecurity': 'Cybersecurity',
    'cloud': 'Cloud',
    
    # Manufacturing & Production
    'production': 'Production',
    'manufacturing': 'Manufacturing',
    'supply chain': 'Supply Chain',
    'logistics': 'Logistics',
    'procurement': 'Procurement',
    
    # Consulting & Advisory
    'consulting': 'Consulting',
    'advisory': 'Advisory',
    'strategy': 'Strategy',
    
    # Other
    'general': 'General',
    'other': 'Other',
    'misc': 'Miscellaneous',
    'miscellaneous': 'Miscellaneous',
}

# Invalid department names to exclude
INVALID_DEPARTMENTS = {
    'na', 'n/a', 'none', 'not specified', 'not mentioned',
    'any', 'all', 'multiple', 'various', 'tbd', 'to be decided'
}


def normalize_department(dept: str) -> Union[str, None]:
    """
    Normalize a single department name.
    
    Args:
        dept: Raw department string
        
    Returns:
        Normalized department name or None if invalid
    """
    if not dept or not isinstance(dept, str):
        return None
    
    # Clean and normalize
    dept = dept.strip().lower()
    
    # Remove common punctuation at edges
    dept = re.sub(r'^[.,;:\-/|()&]+|[.,;:\-/|()&]+$', '', dept).strip()
    
    # Skip if empty or too short
    if not dept or len(dept) < 2:
        return None
    
    # Skip if in invalid list
    if dept in INVALID_DEPARTMENTS:
        return None
    
    # Normalize using mapping
    if dept in DEPARTMENT_NORMALIZATION:
        return DEPARTMENT_NORMALIZATION[dept]
    
    # Return title-cased version if not in mapping
    return dept.title()


def clean_department(value: Union[str, List]) -> List[str]:
    """
    Clean department value by splitting, normalizing, and deduplicating.
    
    Args:
        value: Department string or list
        
    Returns:
        List of unique, normalized department names
        
    Examples:
        "IT/HR/Sales" → ["Information Technology", "Human Resources", "Sales"]
        "Engineering, R&D" → ["Engineering", "Research & Development"]
        "IT, IT, Information Technology" → ["Information Technology"]
    """
    # Handle None, NaN, or empty
    if pd.isna(value) or not value:
        return []
    
    # If already a list
    if isinstance(value, list):
        dept_list = [str(d).strip() for d in value if d]
    else:
        # Convert to string
        value = str(value).strip()
        
        # Remove surrounding brackets if present
        value = re.sub(r'^\[|\]$', '', value)
        
        # Remove quotes
        value = re.sub(r'["\']', '', value)
        
        # Split on multiple separators: comma, slash, hyphen, pipe, ampersand
        # Use regex to split on any of these characters
        dept_list = re.split(r'[,/\-|&]', value)
        
        # Clean each department
        dept_list = [d.strip() for d in dept_list if d and d.strip()]
    
    # Normalize all departments
    normalized = [normalize_department(d) for d in dept_list]
    
    # Remove None values and deduplicate while preserving order
    seen = {}
    for dept in normalized:
        if dept is not None:
            seen[dept] = None
    
    return list(seen.keys())


def get_primary_department(dept_list: List[str]) -> Union[str, None]:
    """Get the first (primary) department from list."""
    return dept_list[0] if dept_list else None


def has_department(dept_list: List[str], target_dept: str) -> bool:
    """Check if target department is in list (case-insensitive)."""
    if not isinstance(dept_list, list):
        return False
    target_normalized = normalize_department(target_dept)
    return target_normalized in dept_list if target_normalized else False


def get_department_count(dept_list: List[str]) -> int:
    """Get count of departments."""
    return len(dept_list) if isinstance(dept_list, list) else 0


# Test cases
test_cases = [
    "IT/HR/Sales",
    "Engineering, R&D",
    "IT, IT, Information Technology",
    "Software Development - Quality Assurance",
    "Sales & Marketing",
    "Finance/Accounts/Admin",
    "DevOps | Cloud | Infrastructure",
    "Product Management, Engineering",
    "NA",
    "Not Specified",
    "",
    None,
    "IT",
    "Human Resources",
    "R&D, Product, Engineering",
    "Data Science/Analytics/AI/ML",
    "[Marketing, Sales]",
    "'IT', 'HR', 'Finance'",
]

print("Testing Department Cleaning:")
print("-" * 100)
for test in test_cases:
    result = clean_department(test)
    primary = get_primary_department(result)
    count = get_department_count(result)
    print(f"Input:   {repr(test):<50}")
    print(f"Output:  {result}")
    print(f"Primary: {primary} | Count: {count}")
    print()

print("=" * 100)
print("Example DataFrame Usage:")
print("=" * 100)

# Example with DataFrame
sample_data = {
    'Department': [
        "IT/HR/Sales",
        "Engineering, R&D",
        "IT, IT, Information Technology",
        "Sales & Marketing",
        "NA",
        "Product Management, Engineering",
        None
    ]
}

df_example = pd.DataFrame(sample_data)

# Apply cleaning
df_example['Department_Cleaned'] = df_example['Department'].apply(clean_department)
df_example['Primary_Department'] = df_example['Department_Cleaned'].apply(get_primary_department)
df_example['Department_Count'] = df_example['Department_Cleaned'].apply(get_department_count)

# Check for specific departments
df_example['Has_IT'] = df_example['Department_Cleaned'].apply(lambda x: has_department(x, 'IT'))
df_example['Has_Engineering'] = df_example['Department_Cleaned'].apply(lambda x: has_department(x, 'Engineering'))

print(df_example[['Department', 'Department_Cleaned', 'Primary_Department', 'Department_Count']])

print("\n" + "=" * 100)
print("Recommended Usage:")
print("=" * 100)
print("""
# Apply department cleaning
df['Department_Cleaned'] = df['Department'].apply(clean_department)
df['Primary_Department'] = df['Department_Cleaned'].apply(get_primary_department)
df['Department_Count'] = df['Department_Cleaned'].apply(get_department_count)

# Filter by department
it_jobs = df[df['Department_Cleaned'].apply(lambda x: has_department(x, 'IT'))]
engineering_jobs = df[df['Primary_Department'] == 'Engineering']

# Department frequency analysis
from collections import Counter
all_depts = [dept for depts in df['Department_Cleaned'] for dept in depts]
dept_frequency = Counter(all_depts)
print(dept_frequency.most_common(20))

# Multi-department jobs
multi_dept_jobs = df[df['Department_Count'] > 1]

# Cross-functional roles (e.g., IT + Sales)
it_sales_jobs = df[df['Department_Cleaned'].apply(
    lambda x: has_department(x, 'IT') and has_department(x, 'Sales')
)]
""")

Testing Department Cleaning:
----------------------------------------------------------------------------------------------------
Input:   'IT/HR/Sales'                                     
Output:  ['Information Technology', 'Human Resources', 'Sales']
Primary: Information Technology | Count: 3

Input:   'Engineering, R&D'                                
Output:  ['Engineering']
Primary: Engineering | Count: 1

Input:   'IT, IT, Information Technology'                  
Output:  ['Information Technology']
Primary: Information Technology | Count: 1

Input:   'Software Development - Quality Assurance'        
Output:  ['Software Development', 'Quality Assurance']
Primary: Software Development | Count: 2

Input:   'Sales & Marketing'                               
Output:  ['Sales', 'Marketing']
Primary: Sales | Count: 2

Input:   'Finance/Accounts/Admin'                          
Output:  ['Finance', 'Administration']
Primary: Finance | Count: 2

Input:   'DevOps | Cloud | Infrastructur

In [23]:
df['Department_Cleaned'] = df['Department'].apply(clean_department)
df['Primary_Department'] = df['Department_Cleaned'].apply(get_primary_department)
df['Department_Count'] = df['Department_Cleaned'].apply(get_department_count)

In [24]:
def clean_timestamp(value: str) -> str:
    """
    Clean the timestamp value by extracting the date part.
    
    Args:
        value: The original timestamp value.
    
    Returns:
        Cleaned date string in 'YYYY-MM-DD' format.
    """
    if not isinstance(value, str):
        return value

    # Split by "T" and take the first part
    date_part = value.split("T")[0]
    
    return date_part

df["Scraped_At_Cleaned"] = df["Scraped_At"].apply(lambda x: clean_timestamp(x))
df["Scraped_At_Cleaned"].sample(10)

3480     2025-12-13
12088    2025-12-19
8243     2025-12-16
291      2025-12-07
5780     2025-12-14
12341    2025-12-19
9551     2025-12-17
7520     2025-12-15
7688     2025-12-15
7745     2025-12-15
Name: Scraped_At_Cleaned, dtype: object

In [25]:
def clean_description(value: str) -> str:
    """
    Clean the job description by removing unwanted characters.
    
    Args:
        value: The original job description.
    
    Returns:
        Cleaned job description string.
    """
    if not isinstance(value, str):
        return value

    # Example cleaning: Remove excessive whitespace
    cleaned_description = ' '.join(value.split())
    
    return cleaned_description

df["Description_Cleaned"] = df["Description"].apply(lambda x: clean_description(x))
df["Description_Cleaned"].sample(10)


14410    Designing and developing new web applications....
6367     Role: Talent Acquisition Specialist Location: ...
7227     About The Role Project Role : Custom Software ...
10359    Job Summary: As a Microsoft Dynamics 365 CRM A...
1674     What Youll Do As a UX Designer, youll work acr...
7503     1) Expert-Level Technical Support: Provide exp...
11310    Job Purpose Skilled Python Developer with 2+ y...
4954     Looking for a QA Engineer who is versed in fun...
9326     Overall 5+ years of experience in the Banking ...
12437    Responsibilities Collaborate with cross-functi...
Name: Description_Cleaned, dtype: object

In [27]:
# storing clean df
df.to_csv(r"D:\DATA SCIENCE AND ML\Project\job_trend_predictor\data\processed\cleaned\cleaned_job_data.csv", index=False)